In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from sklearn.linear_model import LassoCV

from sklearn.metrics import classification_report, confusion_matrix


In [2]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
heart_disease = fetch_ucirepo(id=45)

type(heart_disease)

X = heart_disease.data.features 
y = heart_disease.data.targets 

df_merge=pd.concat([X,y],axis=1)

df_merge.dropna(inplace=True)

In [6]:
df_merge

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
0,63,1,1,145,233,1,2,150,0,2.3,3,0.0,6.0,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3.0,3.0,2
2,67,1,4,120,229,0,2,129,1,2.6,2,2.0,7.0,1
3,37,1,3,130,250,0,0,187,0,3.5,3,0.0,3.0,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0.0,3.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,57,0,4,140,241,0,0,123,1,0.2,2,0.0,7.0,1
298,45,1,1,110,264,0,0,132,0,1.2,2,0.0,7.0,1
299,68,1,4,144,193,1,0,141,0,3.4,2,2.0,7.0,2
300,57,1,4,130,131,0,0,115,1,1.2,2,1.0,7.0,3


In [120]:
df_merge['num'].unique()

array([0, 2, 1, 3, 4])

In [114]:
X_train, X_test, y_train, y_test = train_test_split(df_merge.iloc[:,:-1], df_merge.iloc[:,-1], test_size=0.2, random_state=42)

# Add constant term for intercept
X_train_const = sm.add_constant(X_train)
X_test_const = sm.add_constant(X_test)

In [116]:
# Fit multinomial logit (as proxy for probit)
mnlogit_model = sm.MNLogit(y_train, X_train_const)
mnlogit_result = mnlogit_model.fit()

# Summary
print(mnlogit_result.summary())

# Predict and evaluate
y_pred = mnlogit_result.predict(X_train_const)
y_pred_class = y_pred.idxmax(axis=1)

print("Confusion Matrix:\n", confusion_matrix(y_train, y_pred_class))
print("\nClassification Report:\n", classification_report(y_train, y_pred_class))

         Current function value: 0.789920
         Iterations: 35
                          MNLogit Regression Results                          
Dep. Variable:                    num   No. Observations:                  237
Model:                        MNLogit   Df Residuals:                      181
Method:                           MLE   Df Model:                           52
Date:                Fri, 04 Apr 2025   Pseudo R-squ.:                  0.3933
Time:                        08:04:31   Log-Likelihood:                -187.21
converged:                      False   LL-Null:                       -308.55
Covariance Type:            nonrobust   LLR p-value:                 2.046e-26
     num=1       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -7.3406      3.466     -2.118      0.034     -14.134      -0.547
age           -0.0066      0.028     -0.233      0.816      -0.06

/Users/shreeyeshmenon/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [126]:
y_pred_test = mnlogit_result.predict(X_test_const)
y_pred_class_test = y_pred_test.idxmax(axis=1)


In [130]:
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_class_test))
print("\nClassification Report:\n", classification_report(y_test, y_pred_class_test))

Confusion Matrix:
 [[34  1  0  1  0]
 [ 5  1  0  2  1]
 [ 2  1  1  1  0]
 [ 1  2  1  1  2]
 [ 1  1  0  1  0]]

Classification Report:
               precision    recall  f1-score   support

           0       0.79      0.94      0.86        36
           1       0.17      0.11      0.13         9
           2       0.50      0.20      0.29         5
           3       0.17      0.14      0.15         7
           4       0.00      0.00      0.00         3

    accuracy                           0.62        60
   macro avg       0.32      0.28      0.29        60
weighted avg       0.56      0.62      0.58        60

